In [ ]:
import os
from threading import Lock
from bloom_filter import BloomFilter
from MemTable import MemTable
from WAL import WriteAheadLog
from Segment import SegmentManager


class KVStore:
    """
    A disk-backed key-value store optimized for CRUD operations.
    Includes:
    - Write-Ahead Log for durability
    - MemTable for in-memory writes
    - Segments for efficient disk operations
    """
    def __init__(self, path, bloom_filter_size=100000, error_rate=0.01):
        self.path = path
        self.lock = Lock()
        os.makedirs(path, exist_ok=True)

        # Components
        self.wal = WriteAheadLog(path)
        self.memtable = MemTable()
        self.segment_manager = SegmentManager(path)
        self.bloom = BloomFilter(bloom_filter_size, error_rate=error_rate)
        self.index = {}

        # Start with a fresh segment
        self.current_segment = self.segment_manager.new_segment()

    def set(self, key, value):
        """Set a key-value pair."""
        with self.lock:
            # Log operation
            self.wal.write("SET", key, value)

            # Write to MemTable
            self.memtable.set(key, value)
            self.bloom.add(key)

            # Flush MemTable if needed
            if len(self.memtable.table) >= 1000:
                self.flush_memtable()

    def get(self, key):
        """Retrieve a value by key."""
        with self.lock:
            if key not in self.bloom:
                return None

            # Check MemTable
            value = self.memtable.get(key)
            if value is not None:
                return value

            # Check persistent index
            if key in self.index:
                segment_id, offset = self.index[key]
                return self.segment_manager.segments[segment_id].read(offset)
            return None

    def delete(self, key):
        """Delete a key-value pair."""
        with self.lock:
            # Log operation
            self.wal.write("DEL", key)

            # Remove from MemTable
            self.memtable.delete(key)
            self.bloom.add(key)  # Ensure BloomFilter is consistent

    def flush_memtable(self):
        """Flush the MemTable to the current segment."""
        data = self.memtable.flush()
        for key, value in data.items():
            offset = self.current_segment.write(key, value)
            self.index[key] = (self.current_segment.id, offset)

    def close(self):
        """Close the KVStore and flush data."""
        self.flush_memtable()
        self.segment_manager.close()
        self.wal.file.close()
